In [ ]:
from autumn.projects.sm_covid2.common_school.calibration import get_bcm_object, plot_model_fit

from autumn.projects.sm_covid2.common_school.runner_tools import optimise_model_fit

from scipy.stats import qmc

In [ ]:
iso3 = "FRA"
n_samples = 4

In [ ]:
bcm = get_bcm_object(iso3, "main")

In [ ]:
d = sum([p.size for p in bcm.priors.values()])
sampler = qmc.LatinHypercube(d=d)
regular_sample = sampler.random(n=n_samples)

In [ ]:
l_bounds = [item for sublist in [ [p.bounds()[0]] * p.size for p in bcm.priors.values()] for item in sublist]
u_bounds = [item for sublist in [ [p.bounds()[1]] * p.size for p in bcm.priors.values()] for item in sublist]


In [ ]:
sample = qmc.scale(regular_sample, l_bounds, u_bounds)

In [ ]:
def get_sample_as_dict(s, priors):
    
    i = 0
    sample_dict = {}
    for param_name, p in priors.items():
        if p.size == 1:
            sample_dict[param_name] = s[i]
        else:
            sample_dict[param_name] = list(s[i:(i+p.size)])
        i += p.size
        
    return sample_dict

In [ ]:
sample_as_dicts = [get_sample_as_dict(s, bcm.priors) for s in sample]

In [ ]:
best_params = {}
for j, sample_dict in enumerate(sample_as_dicts):
    best_p, _ = optimise_model_fit(bcm, 2000, 5000, suggested_start=sample_dict)
    best_params[j] = best_p

In [ ]:
for i, params in best_params.items():
    plot_model_fit(bcm, params, f"{iso3}: {i + 1}")